In [1]:
import jax

from quantum_transformers.datasets import get_imdb_dataloaders
from quantum_transformers.training import train_and_evaluate
from quantum_transformers.transformers import Transformer
from quantum_transformers.quantum_layer import get_circuit

data_dir = '/global/cfs/cdirs/m4392/salcc/data'

2023-08-22 06:52:34.279398: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-08-22 06:52:34.279422: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-08-22 06:52:34.279438: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-08-22 06:52:35.412248: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


Instructions for updating:
The TensorFlow Distributions library has moved to TensorFlow Probability (https://github.com/tensorflow/probability). You should update all references to use `tfp.distributions` instead of `tf.distributions`.
Instructions for updating:
The TensorFlow Distributions library has moved to TensorFlow Probability (https://github.com/tensorflow/probability). You should update all references to use `tfp.distributions` instead of `tf.distributions`.


Please first ``pip install -U cirq`` to enable related functionality in translation module


In [2]:
for d in jax.devices():
    print(d, d.device_kind)

gpu:0 NVIDIA A100-SXM4-40GB


In [3]:
(imdb_train_dataloader, imdb_valid_dataloader), vocab, tokenizer = get_imdb_dataloaders(batch_size=32, data_dir=data_dir, max_vocab_size=8192, max_seq_len=128)
print(f"Vocabulary size: {len(vocab)}")
first_batch = next(iter(imdb_train_dataloader))
print(first_batch[0][0])
print(' '.join(map(bytes.decode, tokenizer.detokenize(first_batch[0])[0].numpy().tolist())))

Vocabulary size: 8175
[ 103   99   42  179  112    4    4    4   31  100   18   33   31  100
   18   33   50  178  321  103  112  145  101  498  136   17   50  134
 1214  321  103  112  275   96  101  229 2100  148   98  152 2059   97
  101   17   42  276   97  204  162  400  145  162  163 1368  183   10
   61  414   98 1263  148  145  162  202  192  407  102  207   17   95
  112   99  267   42  179 1186   10   60  584   97   95  685   97  204
  186  188   17 2048   99  123  118  214   15  122   95  257  267  217
   42  146  141  395   96 3172 2370  105  146  141  123 3801   95  155
 4331   95  109  479  411 7227  101   17   95 1250   97   95  112   99
  848  108]
this is a great film ! ! ! < br / > < br / > i first saw this film when it came out . i just recently saw this film again and it still holds up to my memory of it . a lot of films we watched when we were younger don ' t seem to hold up when we watch them later in life . the film is actually a great 80 ' s example of the type 

In [4]:
model = Transformer(num_tokens=len(vocab), max_seq_len=128, num_classes=2, hidden_size=6, num_heads=2, num_transformer_blocks=4, mlp_hidden_size=3,
                    quantum_attn_circuit=get_circuit(), quantum_mlp_circuit=get_circuit())
train_and_evaluate(model, imdb_train_dataloader, imdb_valid_dataloader, num_classes=2, learning_rate=0.0003, num_epochs=30)

Epoch  30/30: 100%|██████████| 781/781 [00:31<00:00, 24.97batch/s, Loss = 0.7142, AUC = 76.48%]

TOTAL TIME = 977.46s
BEST AUC = 76.71% AT EPOCH 23
